# Hey now it is time for part 1 - obtaining and cleaning data

In [1]:
import pandas as pd

In [3]:
pip install lxml

     |████████████████████████████████| 5.8MB 31.2MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


#### Yo I have installed lxml to read frame from wikipedia site

In [2]:
Canada=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

#### Table was under first "Row"

In [3]:
Canada=Canada[0]

In [4]:
Canada.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Let's drop now "Not assigned" from Borougth

In [5]:
Canada.drop(Canada[Canada.Borough=='Not assigned'].index, inplace=True)

In [6]:
Canada.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


#### Now we need to replace "N A" from Neighbourhood to Borough 

In [7]:
Canada.loc[Canada['Neighbourhood'] == 'Not assigned', ['Neighbourhood']]=Canada['Borough']

In [8]:
Canada.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,Queen's Park
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [9]:
Canada.shape

(210, 3)

# Hey, Hey, now it's time for part 2 i.e. assigning longtitude and latitude to the locations

In [11]:
pip install geocoder

     |████████████████████████████████| 102kB 18.4MB/s ta 0:00:01
     |████████████████████████████████| 829kB 33.0MB/s eta 0:00:01
     |████████████████████████████████| 81kB 17.4MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
Note: you may need to restart the kernel to use updated packages.


In [10]:
import geocoder

In [11]:
g = geocoder.google('M5A, Toronto, Ontario')
g


<[REQUEST_DENIED] Google - Geocode [empty]>

#### I had some problems with this geocoder:/ Therefore I've tried to download this csv file

In [12]:
Geo_imp=pd.read_csv('http://cocl.us/Geospatial_data')

In [13]:
Geo_imp.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Done, now I'm merging two pds

In [14]:
Can_ll=pd.merge(Canada, Geo_imp, left_on='Postcode', right_on='Postal Code', how='left')

In [15]:
Can_ll.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,M5A,43.654260,-79.360636
3,M6A,North York,Lawrence Heights,M6A,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,M6A,43.718518,-79.464763


##### I can drop "Postal Code" cos it's doubled

In [16]:
Can_ll=Can_ll.drop('Postal Code', 1)

In [17]:
Can_ll.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Heights,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,43.718518,-79.464763


# Now we are heading to part no. 3 - analysis 

In [18]:
Tor_ll=Can_ll[Can_ll['Borough'].str.contains('Toronto')]

In [19]:
Tor_ll.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
12,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
13,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
26,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
35,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [20]:
Tor_ll.shape

(73, 5)

In [21]:
Tor_ll=Tor_ll.reset_index(drop=True)

In [22]:
Tor_ll.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
2,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


### Now I'm importing all the stuff from NY workshop

In [24]:
import numpy as np

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 

from geopy.geocoders import Nominatim

import requests

from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

print('Libraries imported.')

Libraries imported.


In [25]:
print('It seems that Toronto has {} boroughs and {} neighborhoods.'.format(
        len(Tor_ll['Borough'].unique()),
        len(Tor_ll['Neighbourhood'].unique())
    )
)

It seems that Toronto has 4 boroughs and 72 neighborhoods.


In [42]:
CLIENT_ID = 'blah blha blah' # your Foursquare ID
CLIENT_SECRET = 'blah blha blah' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID:' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID:blah blha blah
CLIENT_SECRET:blah blha blah


In [27]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### I've just Ctrl+c, Ctrl+v two functions to get all data for each neighburhood

In [29]:
LIMIT=100

In [30]:
Toronto_venues = getNearbyVenues(names=Tor_ll['Neighbourhood'],
                                   latitudes=Tor_ll['Latitude'],
                                   longitudes=Tor_ll['Longitude']
                                  )

Harbourfront
Ryerson
Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
The Danforth West
Riverdale
Design Exchange
Toronto Dominion Centre
Brockton
Exhibition Place
Parkdale Village
The Beaches West
India Bazaar
Commerce Court
Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North
Forest Hill West
High Park
The Junction South
North Toronto West
The Annex
North Midtown
Yorkville
Parkdale
Roncesvalles
Davisville
Harbord
University of Toronto
Runnymede
Swansea
Moore Park
Summerhill East
Chinatown
Grange Park
Kensington Market
Deer Park
Forest Hill SE
Rathnelly
South Hill
Summerhill West
CN Tower
Bathurst Quay
Island airport
Harbourfront West
King and Spadina
Railway Lands
South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
St. James Town
First Canadian Place
Underground city
Church and W

#### Quita  lot of stuff, let's check how many venues were read. I've set limit = 100 an I keep it in mind

In [31]:
print(Toronto_venues.shape)
Toronto_venues.head()


(3225, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [32]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Bathurst Quay,16,16,16,16,16,16
Berczy Park,58,58,58,58,58,58
Brockton,21,21,21,21,21,21
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
...,...,...,...,...,...,...
Underground city,100,100,100,100,100,100
Union Station,100,100,100,100,100,100
University of Toronto,37,37,37,37,37,37


#### There are some Nbh with 100 venues, which is bad news, but anyway, this workshop is only for show off. Now I'm getting dummies

In [33]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

#there were some collumn with "Ngh, so i've dropped it"
Toronto_onehot = Toronto_onehot.drop('Neighborhood', 1)

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Adelaide,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.030000,0.0,...,0.0,0.0,0.00,0.020000,0.000000,0.0,0.01,0.0,0.0,0.000000
1,Bathurst Quay,0.0,0.0625,0.0625,0.0625,0.125,0.1875,0.0625,0.000000,0.0,...,0.0,0.0,0.00,0.000000,0.000000,0.0,0.00,0.0,0.0,0.000000
2,Berczy Park,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,0.0,...,0.0,0.0,0.00,0.017241,0.000000,0.0,0.00,0.0,0.0,0.000000
3,Brockton,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,0.0,...,0.0,0.0,0.00,0.000000,0.000000,0.0,0.00,0.0,0.0,0.000000
4,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,0.0,...,0.0,0.0,0.00,0.000000,0.000000,0.0,0.00,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,Underground city,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.030000,0.0,...,0.0,0.0,0.01,0.010000,0.000000,0.0,0.01,0.0,0.0,0.000000
68,Union Station,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,0.0,...,0.0,0.0,0.01,0.010000,0.000000,0.0,0.01,0.0,0.0,0.000000
69,University of Toronto,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,0.0,...,0.0,0.0,0.00,0.000000,0.027027,0.0,0.00,0.0,0.0,0.027027
70,Victoria Hotel,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.040000,0.0,...,0.0,0.0,0.00,0.020000,0.000000,0.0,0.01,0.0,0.0,0.000000


In [35]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.05
2       Steakhouse  0.04
3              Bar  0.04
4  Thai Restaurant  0.04


----Bathurst Quay----
                 venue  freq
0      Airport Service  0.19
1       Airport Lounge  0.12
2  Rental Car Location  0.06
3              Airport  0.06
4          Coffee Shop  0.06


----Berczy Park----
         venue  freq
0  Coffee Shop  0.07
1       Bakery  0.05
2         Café  0.03
3  Cheese Shop  0.03
4   Steakhouse  0.03


----Brockton----
                   venue  freq
0         Breakfast Spot  0.10
1                   Café  0.10
2            Coffee Shop  0.10
3      Convenience Store  0.05
4  Performing Arts Venue  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.12
1                 Spa  0.06
2          Comic Shop  0.06
3                Park  0.06
4    Recording Studio  0.06


----CN Tower----
                 venue  freq
0     

In [36]:
# set number of clusters - i've decided to do 7 clusters lol - 5 was nasty
kclusters = 7

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 4, 3, 3, 3, 4, 3, 3, 3, 3], dtype=int32)

In [37]:
Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)
Toronto_grouped_clustering

,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.030000,0.0,0.00,...,0.0,0.0,0.00,0.020000,0.000000,0.0,0.01,0.0,0.0,0.000000
1,0.0,0.0625,0.0625,0.0625,0.125,0.1875,0.0625,0.000000,0.0,0.00,...,0.0,0.0,0.00,0.000000,0.000000,0.0,0.00,0.0,0.0,0.000000
2,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,0.0,0.00,...,0.0,0.0,0.00,0.017241,0.000000,0.0,0.00,0.0,0.0,0.000000
3,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,0.0,0.00,...,0.0,0.0,0.00,0.000000,0.000000,0.0,0.00,0.0,0.0,0.000000
4,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,0.0,0.00,...,0.0,0.0,0.00,0.000000,0.000000,0.0,0.00,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.030000,0.0,0.00,...,0.0,0.0,0.01,0.010000,0.000000,0.0,0.01,0.0,0.0,0.000000
68,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,0.0,0.05,...,0.0,0.0,0.01,0.010000,0.000000,0.0,0.01,0.0,0.0,0.000000
69,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,0.0,0.00,...,0.0,0.0,0.00,0.000000,0.027027,0.0,0.00,0.0,0.0,0.027027
70,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.040000,0.0,0.00,...,0.0,0.0,0.00,0.020000,0.000000,0.0,0.01,0.0,0.0,0.000000


#### Hey, I'm creating new dataframe with 10 most popular venues

In [38]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [39]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Thai Restaurant,Steakhouse,Bar,American Restaurant,Sushi Restaurant,Restaurant,Burger Joint,Cosmetics Shop
1,Bathurst Quay,Airport Service,Airport Lounge,Harbor / Marina,Boutique,Bar,Coffee Shop,Sculpture Garden,Boat or Ferry,Rental Car Location,Airport Terminal
2,Berczy Park,Coffee Shop,Bakery,Cheese Shop,Cocktail Bar,Steakhouse,Café,Farmers Market,Beer Bar,Seafood Restaurant,Basketball Stadium
3,Brockton,Coffee Shop,Café,Breakfast Spot,Grocery Store,Intersection,Pet Store,Gym,Climbing Gym,Restaurant,Sandwich Place
4,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Farmers Market,Auto Workshop,Comic Shop,Pizza Place,Recording Studio,Restaurant,Burrito Place,Brewery,Skate Park


In [40]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Tor_ll

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

Toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,3,Coffee Shop,Park,Bakery,Pub,Café,Mexican Restaurant,Restaurant,Breakfast Spot,Theater,Farmers Market
1,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937,3,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Café,Cosmetics Shop,Fast Food Restaurant,Bakery,Bookstore,Restaurant,Sporting Goods Shop
2,M5B,Downtown Toronto,Garden District,43.657162,-79.378937,3,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Café,Cosmetics Shop,Fast Food Restaurant,Bakery,Bookstore,Restaurant,Sporting Goods Shop
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,3,Coffee Shop,Restaurant,Café,Italian Restaurant,Bakery,Park,Diner,Hotel,Clothing Store,Breakfast Spot
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Health Food Store,Pub,Trail,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


In [41]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# This is it - it seems that neighbours in Toronto are sort of similar. Only outliers are different